In [1]:
from config import *
import random
from glob import glob
import concurrent.futures
from tqdm import tqdm as tqdm
import numpy as np
import pandas as pd
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import math
import tensorflow as tf
plt.rcParams['axes.grid'] = False
# plt.rcParams['figure.figsize'] = 

time zone: US/Eastern


# Preprocessing

We will be doing two types of preprocessing. First will be the preprocessing done only once and the second will be the preprocessing done on-the-fly using the tensorflow's Dataset API. 

Our goals in this notebook are:
1. To normalize the lighting conditions across photos.
2. To normalize crop conditions across photos.
3. To discern if the photo is from left or right eye.
4. Identify landmarks in the photo.
    * Bright spot
    * Edge
    * Veins

In [2]:
# brightnesses = load_pickle(os.path.join(DUMP_DIR, 'brightnesses.pkl'))
# temp = {}
# for ds in brightnesses:
#     temp[ds] = []
#     for m, p in brightnesses[ds]:
#         temp[ds].append((m, p.replace('/media/Projects/kaggle', '/home/kdonbekci')))
# brightnesses = temp

In [3]:
paths, labels = {}, {}
paths['aptos'] = {'train': glob(os.path.join(DATA_DIR, 'aptos', 'train', 'img', '*.png'))}
paths['chf'] = {'train': glob(os.path.join(DATA_DIR,'chf' , 'train', 'img', '*.jpeg')),
                'test': glob(os.path.join(DATA_DIR, 'chf', 'test', 'img', '*.jpeg'))}
# paths['idrid'] = glob(os.path.join(DATA_DIR, 'idrid', 'grading', 'train', 'img', '*.jpg'))
# paths['idrid'] += glob(os.path.join(DATA_DIR,'idrid', 'grading', 'test', 'img', '*.jpg'))
labels['aptos'] = {'train': pd.read_csv(os.path.join(DATA_DIR, 'aptos', 'train', 'labels.csv')).sort_values(by='id')}
labels['chf'] =  {'train': pd.read_csv(os.path.join(DATA_DIR, 'chf', 'train', 'labels.csv')).sort_values(by='id'), 
                  'test': pd.read_csv(os.path.join(DATA_DIR, 'chf', 'test', 'labels.csv')).sort_values(by='id')}
for key in paths:
    for split in paths[key]:
        paths[key][split].sort()

### 1. Reading Image
First step is to read in the image and make a grascale copy.

In [4]:
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

### 2. Finding the Eye

In [5]:
def find_eye(im):
    cy = im.shape[0]//2
    midline = im[cy,:]
    midline = np.where(midline>midline.mean()/3)[0]
    if len(midline)>im.shape[1]//2:
        x_start, x_end = np.min(midline), np.max(midline)
    else: # This actually rarely happens p~1/10000
        x_start, x_end = im.shape[1]//10, 9*im.shape[1]//10
    cx = (x_start + x_end)/2
    r = (x_end - x_start)/2
    return cx, cy, r

### 3. Applying Ben's Color Correction

In [7]:
def subtract_median_bg_image(img):
    k =np.max(img.shape)//20*2+1
    bg = cv2.medianBlur(img, k)
    return cv2.addWeighted(img, 4, bg, -4, 128)

In [8]:
def color_correct(img):
    h, w, _ = img.shape
    sigmaX = min(h, w)/30
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0,0), sigmaX), -4 ,128)
    return img

### 4. Image Resizing
I will try a different approach on resizing. Instead of lowering the pixel count and reducing information, I will attempt to take several crops of the full-sized image in my desired image size.

In [9]:
def resize_image(img, size):
    cx, cy, r = find_eye(img)
    scaling = size/(2*r)
    rotation = 0
    M = cv2.getRotationMatrix2D((cx,cy), rotation, scaling)
    M[0,2] -= cx - size/2
    M[1,2] -= cy - size/2
    return cv2.warpAffine(img,M,(size,size)) # This is the most important line

In [10]:
def radius_reduce(img, param=96):
    h,w,c=img.shape
    Frame=np.zeros((h,w,c),dtype=np.uint8)
    cv2.circle(Frame,(int(math.floor(w/2)),int(math.floor(h/2))),int(math.floor((h*param)/float(2*100))), (255,255,255), -1)
    Frame1=cv2.cvtColor(Frame, cv2.COLOR_BGR2GRAY)
    img1 =cv2.bitwise_and(img,img,mask=Frame1)
    return img1

### Putting it all together

In [11]:
SIZE = 224
per_chunk = 2048

In [12]:
def preprocess(path, size):
    img = read_image(path)
    img = resize_image(img, size)
    img = subtract_median_bg_image(img)
    img = radius_reduce(img)
    return img

In [13]:
def serialize(img, label, _id):
    feature = {
        'img': bytes_feature(cv2.imencode('.jpg', img)[1].tostring()),
        'diagnosis': int64_feature(label),
        'id': bytes_feature(_id.encode())
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [14]:
def preprocess_and_serialize(d):
    path, diagnosis, _id = d
    img = preprocess(path, SIZE)
    return serialize(img, diagnosis, _id)

In [15]:
data = []
# for ds in paths:
ds = 'chf'
for split in paths[ds]:
    grades = np.asarray(labels[ds][split].diagnosis)
    ids = np.asarray(labels[ds][split].id)
    for i, path in enumerate(tqdm(paths[ds][split])):
        data.append((path, grades[i], ids[i]))
random.shuffle(data)

100%|██████████| 53576/53576 [00:00<00:00, 1087834.78it/s]


In [16]:
%%time
serialized = []
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    serialized = list(executor.map(preprocess_and_serialize, data))

Process Process-3:
Traceback (most recent call last):
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/concurrent/futures/process.py", line 181, in _process_worker
    result=r))
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/queues.py", line 347, in put
    self._writer.send_bytes(obj)
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
  File "/home/kdonbekci/anaconda3/envs/aptos/lib/python3.6/multiprocessing/connection.py", line 368,

KeyboardInterrupt: 

In [ ]:
num_chunks = int(np.ceil(len(serialized)/per_chunk))
for i in range(num_chunks):
    record_file = '{}_{:02d}.tfrecords'.format(ds, i)
    with tf.io.TFRecordWriter(record_file) as writer:
        for s in serialized[i*per_chunk : (i+1)*per_chunk]:
            writer.write(s)